<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/example_kaggle/les_12/les_12_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://www.tensorflow.org/text/tutorials/text_generation

https://www.tensorflow.org/text/tutorials/nmt_with_attention

https://www.tensorflow.org/text/tutorials/transformer


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf


# Own transformer

In [2]:
!pip install tf-models-official

In [3]:
import tensorflow_models as tfm

In [4]:
t = tf.random.normal(shape=(1, 3, 4))
display(t)

t_layer = tfm.nlp.layers.Transformer(num_attention_heads=4, intermediate_size=4*4, intermediate_activation='relu')
display(t_layer(t))


<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
array([[[-0.21696867, -0.33606952, -0.553795  , -1.7197393 ],
        [ 1.3715242 , -0.6138962 , -0.9699648 , -0.70717895],
        [ 0.07083237, -0.5093332 , -0.15982734, -1.9293455 ]]],
      dtype=float32)>

<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
array([[[ 0.4811978 , -0.44003242,  1.3161994 , -1.3573647 ],
        [ 1.6150346 , -0.76993924, -0.8925285 ,  0.0474331 ],
        [ 0.24523357, -0.47611645,  1.4731086 , -1.2422258 ]]],
      dtype=float32)>

In [5]:
for sm in t_layer.submodules:
  if sm.name not in ['key', 'attention_output', 'query', 'value']:
    prefix = f'{t_layer.name}/{sm.name}/'
    print(sm.name)
    if 'weights' in dir(sm):
      for w in sm.weights:
        print(f'    {w.name[len(prefix):]}', '\t\t\t', w.shape)
    else:
      print(f'   ', sm.submodules)

dropout
self_attention
    query/kernel:0 			 (4, 4, 1)
    query/bias:0 			 (4, 1)
    key/kernel:0 			 (4, 4, 1)
    key/bias:0 			 (4, 1)
    value/kernel:0 			 (4, 4, 1)
    value/bias:0 			 (4, 1)
    attention_output/kernel:0 			 (4, 1, 4)
    attention_output/bias:0 			 (4,)
self_attention_layer_norm
    gamma:0 			 (4,)
    beta:0 			 (4,)
dropout_1
activation
intermediate
    kernel:0 			 (4, 16)
    bias:0 			 (16,)
output
    kernel:0 			 (16, 4)
    bias:0 			 (4,)
dropout_2
output_layer_norm
    gamma:0 			 (4,)
    beta:0 			 (4,)
dropout_3
softmax


# Донавчання - Transfer Learning

- Bert

- GPT

- T5

## Prompt Engineering

- Context
- Task
- Instruction
- Clarity
- Refine

# GPT 2

In [7]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, GPT2LMHeadModel

model_name = 'gpt2-large'  # gpt2 gpt2-medium gpt2-large

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
GPT2 = TFGPT2LMHeadModel.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

GPT2.summary()

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  774030080 
 er)                                                             
                                                                 
Total params: 774030080 (2.88 GB)
Trainable params: 774030080 (2.88 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
input_sequense = "After coming to the work I"

prompt1 = """
  2+2=4
  16+5=21
  20+3=[endprompt]
  """

prompt2 = """
  Taras Shevchenko: Kaniv
  Ivan Franko: Lviv
  Lesea Ukrayinka:
  """

input_ids = tokenizer.encode(input_sequense, return_tensors='tf')

output = GPT2.generate(input_ids, max_length=30, num_beams=1, do_sample=True, temperature=1, num_return_sequences=1)


In [13]:
for out in tokenizer.batch_decode(output, skip_special_tokens=True):
  print(out)
  print('---------')

After coming to the work I asked, "What have I ever been?" And he said (to him): "A shepherd in the sheepherd
---------


In [14]:
from transformers import pipeline, set_seed
from torch import tensor
generator = pipeline('text-generation', model='gpt2', tokenizer=tokenizer)
set_seed(0)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [25]:
prompt_sentiment = """Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment:"""

prompt_addition = """Addition
Numbers: 2,2
Result: 4
###
Numbers: 16,5
Result: 21
###
Numbers: 41,1
Result:"""

prompt2 = """Burial city
###
Taras Shevchenko: Kaniv
###
Ivan Franko: Lviv
###
Lesea Ukrayinka:"""

# result = generator(prompt_sentiment, top_k=2, temperature=1, max_length=55)
# result = generator(prompt_addition, top_k=2, temperature=1, max_length=55)
result = generator(prompt2, top_k=2, temperature=1, max_length=55)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=55) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [21]:
print(result[0]['generated_text'])

Burial city
Taras Shevchenko: Kaniv
###
Ivan Franko: Lviv
###
Lesea Ukrayinka: Lviv
###
Kelsey Kostenko: Lviv
##
###
###
###
###
###
###
##
###
##
###
##
##
###
##
###
##
##
###
##
###
###
###
###
###
###
###
###
###
###
###
###
###
###
###
####
####
####
#################################################################################################################################################################################################################################################################################################################################


In [31]:
to_summarize = """When I am dead, bury me
In my beloved Ukraine,
My tomb upon a grave mound high
Amid the spreading plain,
So that the fields, the boundless steppes,
The Dnieper's plunging shore
My eyes could see, my ears could hear
The mighty river roar."""

result = generator(
    f"""Summarization Task:\n{to_summarize}\nTL;DR:""",
    max_length=512,
    top_k=5,
    temperature=1,
    no_repeat_ngram_size=2
)

print(result[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summarization Task:
When I am dead, bury me
In my beloved Ukraine,
My tomb upon a grave mound high
Amid the spreading plain,
So that the fields, the boundless steppes,
The Dnieper's plunging shore
My eyes could see, my ears could hear
The mighty river roar.
TL;DR: The Ukraine is a place where you are not allowed to die
You have a choice. You are either to be buried in the ground, or you die in a field. The Ukrainian has no choice, and the Ukrainian is dead. If you choose to bury yourself, you will die as a dead person, but if you decide to live as dead people, then it will be your choice to do the same. So, in this case I choose the burial method, which I will call the "Ukrainian burial".
Ukrautians are a very special kind of people. They have been living in Ukraine for over 100 years. In fact, I was born in Kiev in 1991. I had lived in Crimea for a long time and then I moved to Kiev to start my Ukrainian studies in 1993. My mother lived with me and my father in St. Petersburg. We live